## Homework 03-vector-search

In [1]:
import requests 
from tqdm.auto  import tqdm

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

### Q1. Getting the embeddings model

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"
model.encode(user_question)[0]

0.078222625

In [4]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
len(documents)

948

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]

In [8]:
len(documents)

375

### Q2. Creating the embeddings

In [9]:
embeddings = []
for doc in documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    text_vector = model.encode(qa_text).tolist()
    embeddings.append(text_vector)
    
X = np.array(embeddings)

In [10]:
X.shape

(375, 768)

### Q3. Search

In [11]:
v = model.encode(user_question)
scores = X.dot(v)

In [12]:
max(scores)

0.6506574805426737

### Vector search

In [13]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [14]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
results = search_engine.search(v, num_results=5)

for result in results:
    print('Question: ', result['question'])
    print('Result: ', result['text'], '\n')

Question:  The course has already started. Can I still join it?
Result:  Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate. 

Question:  I just joined. What should I do next? How can I access course materials?
Result:  Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.
Click on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.
Or you can just use this link: http://mlzoomcamp

### Q4. Hit-rate for our search engine

In [15]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [16]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [17]:
results[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id'])

In [18]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [19]:
def question_vector_search_engine(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [20]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [21]:
evaluate(ground_truth, question_vector_search_engine)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

### Q5. Indexing with Elasticsearch

In [22]:
documents = docs_response.json()
course = "machine-learning-zoomcamp"
documents = [doc for doc in documents if doc['course'] == course]

In [23]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/375 [00:00<?, ?it/s]

In [24]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id', 'question_vector', 'text_vector', 'question_text_vector'])

In [25]:
for vector in ['question_vector', 'text_vector', 'question_text_vector']:
    print(vector, ':', len(documents[0][vector]))

question_vector : 768
text_vector : 768
question_text_vector : 768


In [26]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [28]:
def elastic_search(vector):
    search_query = {
        "field": "question_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    es_results = es_client.search(
        index=index_name,
        knn=search_query,
        source=["text", "section", "question", "course", "id"]
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [29]:
user_question = "I just discovered the course. Can I still join it?"
v_q = model.encode(user_question)
elastic_search(vector=v_q)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

### Q6. Hit-rate for Elasticsearch

In [30]:
def elastic_question_vector_search_engine(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)
    return elastic_search(v_q)

In [31]:
evaluate(ground_truth, elastic_question_vector_search_engine)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.8076502732240437}